In [ ]:
# plotting
fig, (ax0, ax1, ax2, ax3, ax4, ax5) = plt.subplots(6, 1, figsize=(8, 10))

ax0.set_title("original")
ax0.plot(original.history["total_frobenius_norm"]/np.max(original.history["total_frobenius_norm"]) * np.max(original.history["val_loss"]))
ax0.plot(original.history["val_loss"])
ax0.plot(original.history["loss"])
ax0.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax0.savefig("convnet_original")
# ax0.show()

ax1.set_title("L1")
ax1.plot(lassoReg.history["total_frobenius_norm"]/np.max(lassoReg.history["total_frobenius_norm"]) * np.max(lassoReg.history["val_loss"]))
ax1.plot(lassoReg.history["val_loss"])
ax1.plot(lassoReg.history["loss"])
# ax1.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax1.savefig("convnet_L1")
# ax1.show()

ax2.set_title("L2")
ax2.plot(ridgeReg.history["total_frobenius_norm"]/np.max(ridgeReg.history["total_frobenius_norm"]) * np.max(ridgeReg.history["val_loss"]))
ax2.plot(ridgeReg.history["val_loss"])
ax2.plot(ridgeReg.history["loss"])
# ax2.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax2.savefig("convnet_L2")
# ax2.show()

ax3.set_title("L2")
ax3.plot(augment.history["total_frobenius_norm"]/np.max(augment.history["total_frobenius_norm"]) * np.max(augment.history["val_loss"]))
ax3.plot(augment.history["val_loss"])
ax3.plot(augment.history["loss"])
# ax3.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax3.savefig("convnet_augment")
# ax3.show()

ax4.set_title("Dropout")
ax4.plot(dropout.history["total_frobenius_norm"]/np.max(dropout.history["total_frobenius_norm"]) * np.max(dropout.history["val_loss"]))
ax4.plot(dropout.history["val_loss"])
ax4.plot(dropout.history["loss"])
# ax4.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax4.savefig("convnet_dropout")
# ax4.show()

ax5.set_title("Batch Norm")
ax5.plot(batchnorm.history["total_frobenius_norm"]/np.max(batchnorm.history["total_frobenius_norm"]) * np.max(batchnorm.history["val_loss"]))
ax5.plot(batchnorm.history["val_loss"])
ax5.plot(batchnorm.history["loss"])
# ax5.legend(labels=["Total Frobenius Norm", "Validation Loss", "Loss"])
# ax5.savefig("convnet_batchnorm")
# ax5.show()

ax5.set_xlabel("Epochs")
# ax5.set_ylabel("Loss")

plt.legend()
fig.savefig("5 optimization methods comparison (e=15)")

fig.tight_layout(pad=0.1)
plt.show()

In [ ]:
# a special preprocess step for applying data augmentation
def preprocess_aug(dataset, augmentation):

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

    # create one-hot targets with depth 10 since cifar 10 has 10 classes
    dataset = dataset.map(lambda img, target: (img, tf.one_hot(target, depth=10)))

    # cache
    dataset = dataset.cache()

    # shuffle, batch, prefetch
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(32)

    # augmentation_model = tf.keras.Sequential([keras_cv.layers.RandAugment(value_range=[0,1],magnitude=0.1)])

    dataset = dataset.map(lambda x, y : (augmentation_model(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    # return preprocessed dataset
    return dataset

In [ ]:
# (not working yet) Option 6 - Data augmentation (**train_dataset will be overwritten if you run this**)

augmentation_model = tf.keras.Sequential([keras_cv.layers.RandAugment(value_range=[0,1],magnitude=0.1)])
train_dataset = preprocess_aug(train_ds, augmentation=augmentation_model) # it overwrites train_dataset=preprocess(train_ds)
network = ConvModel()